# Estimating Gentrification using Street View Images and Embeddings

This script (initially produced by ChatGPT) does the following (_this was my query_):
 - Read a spatial boundary file (that I will hard code)
 - Obtain the road network (from OSM?) for that area
 - Generate sample points on the road network roughly X meters apart
 - At each sample point, download the most recent street images for that location (either a single 360 degree view of a few smaller images). Use whichever API service is the most appropriate for obtaining the images. Importantly please record the date that the image was taken.
 - For each image, calculate an embedding using an appropriate foundation model (one that has been pre-trained to distinguish street environments specifically). Please use Hugging Face libraries.
 - If necessary, calculate the mean embedding for each point (is this the best way to calculate a single embedding for a point represented by multiple images?)
 - Now, for each sampled point there will be a dataframe with information about the point and its embedding. Read another polygon spatial data file, that I will provide, which contains area-level estimates of gentrification.
 - Use point-in-polygon to get the gentrification for each point.
 - Use cross-validation to train a couple of ML models (probaly random forest, linear regression and a neural network) to estimate gentrification from the embedding vectors
 - Choose the best model and parameter configuration and test this model on some held-out data.

## Configuration and library loading

In [1]:
import os
os.environ["HF_HOME"] = "/nfs/a319/gy17m2a/scratch/hf_cache"
import random
import pickle
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import osmnx as ox
import torch
from PIL import Image
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from scipy.stats import entropy
import clip
import pandas as pd

# # Hugging Face Transformers for image embedding
from transformers import AutoImageProcessor, AutoModel, ViTModel, ViTImageProcessor, ViTConfig

# # ----------------- Configuration -----------------
# #np.random.seed(42)
# density_per_km = 0.3  # number of points to sample per km of road
# #density_per_km = 0.1  # VERY FEW WHILE TESTING
DOWNLOAD_IMAGES = False  # If false then don't download any images, just load those that have been cached

data_dir = os.path.join("../../../data/embeddings/")
boundary_file = os.path.join(data_dir, "greater_manchester_lsoas.geojson")  # Path to boundary polygon file
lsoas_file = os.path.join("../../../data/SpatialData/", "LSOAs_2021", "LSOA_2021_EW_BSC_V4.shp")
n_directions = 4         # number of street view images per point (e.g., 4 cardinal directions)

## Data Loading

In [2]:
# --- Load neighbourhood polygons and dissolve to one study-area boundary ---
boundary_neighs = gpd.read_file(boundary_file)
# Make sure we're in WGS84 (lat/lon) for OSM and APIs
boundary_neighs = boundary_neighs.to_crs(epsg=4326)
# Dissolve: merge all geometries into one polygon (MultiPolygon possible)
boundary_polygon = boundary_neighs.union_all()  # shapely (multi)polygon
boundary_gdf = gpd.GeoDataFrame(data={'name': ['study_area']},
    geometry=[boundary_polygon], crs=boundary_neighs.crs)
print("Merged neighbourhoods into single study-area boundary.")
print("Bounds:", boundary_polygon.bounds)
# boundary_gdf.plot(color='lightblue', edgecolor='black')

Merged neighbourhoods into single study-area boundary.
Bounds: (-2.73052481406758, 53.3281053809015, -1.90962093258169, 53.6857339236763)


## Read the LSOA boundary data

(later it will be joined to the Greater Manchester Gentrification Index and IMD)

In [3]:
lsoas =  gpd.read_file(lsoas_file)
manc_lads = ['Manchester', 'Rochdale', 'Bolton', 'Bury', 'Wigan', 'Oldham',  'Trafford', 'Salford', 'Tameside', 'Stockport']
manc_lads_pattern = '|'.join(manc_lads)
gm_lsoa=lsoas[lsoas['LSOA21NM'].str.contains(manc_lads_pattern)]
gm_lsoa = gm_lsoa.to_crs(epsg=4326)
# gm_lsoa.plot()

### Load list of sample points

In [4]:
# Cache file for the entire points data with embeddings (images are stored separately)
DEBUG = False
points_data_cache = data_dir + "sample_points_cache/points_data_cache_with_embeddings.pkl"

# -----------------------------------------------------------
# Load existing cache so we can *append* new sample points
# -----------------------------------------------------------
if os.path.isfile(points_data_cache):
    print("Loading cached point data …")
    with open(points_data_cache, "rb") as f:
        point_records = pickle.load(f)
    existing_coords = {(rec["latitude"], rec["longitude"]) for rec in point_records}
    next_id = max(rec["point_id"] for rec in point_records) + 1
else:
    point_records = []
    existing_coords = set()
    next_id = 0

print(f"Cache currently has {len(point_records)} points.")
added_this_run = 0

Loading cached point data …
Cache currently has 18897 points.


## Compute the Embeddings

(Note: would like to use Places365 but not available in Hugging Face yet, so using ViT base model instead)

In [14]:
device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# ---- Create final text embeddings ----
final_text_features = []
category_names = []  # so your categories keep their correct order

# multi_prompts = {
#     "modern": [
#         "a photo showing a modern style",
#         "a clean contemporary aesthetic",
#         "modern architecture or interior design",
#         "a sleek, newly renovated modern look"
#     ],
#     "traditional": [
#         "a photo with a traditional style",
#         "classic older architecture or interiors",
#         "traditional British housing or decor",
#         "a place with a historic or vintage aesthetic"
#     ],
#     "affluent": [
#         "a photo that looks affluent or wealthy",
#         "a well-maintained high-end environment",
#         "a stylish, expensive-looking area",
#         "a photo suggesting wealth or prosperity"
#     ],
#     "run_down": [
#         "a photo showing a run-down environment",
#         "signs of wear, damage, or poor maintenance",
#         "a neglected or deteriorating place",
#         "a photo suggesting disrepair or decay"
#     ],
#     "minimalist": [
#         "a minimalist design style",
#         "clean simple lines and sparse decoration",
#         "a clutter-free modern interior",
#         "a calm minimalist aesthetic"
#     ],
#     "cluttered": [
#         "a cluttered messy-looking scene",
#         "an interior with many scattered objects",
#         "a busy, chaotic environment",
#         "a photo showing visual clutter"
#     ],
#     "renovated": [
#         "a newly renovated or refurbished interior",
#         "recently upgraded modern features",
#         "a fresh high-quality renovation",
#         "a renewed modernised environment"
#     ],
#     "dated": [
#         "a dated or old-fashioned decor style",
#         "an interior with older outdated features",
#         "a place that looks stuck in the past",
#         "a photo with an out-of-date aesthetic"
#     ]
# }


multi_prompts = {
    "street_trees": [
        "a street scene with visible street trees",
        "a road lined with trees",
        "trees planted along an urban street",
        "a city street with noticeable tree canopy"
    ],

    "greenery_quality": [
        "a scene with healthy green vegetation",
        "a photo showing well-maintained grass or plants",
        "lush greenery in an urban environment",
        "green plants or trees that appear healthy and vibrant"
    ],

    "building_condition": [
        "a photo of a building with visible wear or decay",
        "a street scene showing weathered or poorly maintained buildings",
        "a building with cracked paint, damage, or ageing materials",
        "an older or run-down building facade"
    ],

    "surface_quality": [
        "a street or pavement with potholes or cracks",
        "a damaged sidewalk or road surface",
        "a photo of worn asphalt or uneven pavement",
        "poorly maintained street or footpath surfaces"
    ],

    "street_cleanliness": [
        "a street scene with visible litter or rubbish",
        "an outdoor area that looks unclean or poorly maintained",
        "a city street with scattered trash",
        "an urban scene showing litter or waste on the ground"
    ],

    "pedestrian_infrastructure": [
        "a street scene with clear pedestrian infrastructure",
        "a road with sidewalks, footpaths or crossings",
        "a city street designed for pedestrians",
        "an urban pedestrian-friendly environment"
    ],

    "lighting_and_visibility": [
        "a street with streetlights or lighting infrastructure",
        "a scene showing lamps or artificial lighting",
        "urban streetlight poles visible in the environment",
        "a road at dusk with visible lighting structures"
    ],

    "parking_pressure": [
        "a street scene dominated by parked cars",
        "cars tightly parked along a residential street",
        "a street with heavy on-street parking",
        "roadside parking filling most of the street view"
    ],

    "green_open_space": [
        "a public green open space such as a park or field",
        "a large grassy area in a neighbourhood",
        "an open green parkland scene",
        "a public field or recreational green area"
    ],

    "street_furniture": [
        "a street scene with benches, bins or bollards",
        "urban street furniture visible on a sidewalk",
        "public seating or bins along a footpath",
        "a street with bollards, benches or other fixtures"
    ],

    "graffiti_and_vandalism": [
        "a wall or structure with graffiti or vandalism",
        "a building or street surface marked with graffiti",
        "tagged walls or vandalised public infrastructure",
        "a scene showing graffiti or street tagging"
    ]
}

with torch.no_grad():
    for cat, prompts in multi_prompts.items():
        category_names.append(cat)

        # Encode all prompts for this category
        tokenized = clip.tokenize(prompts).to(device)
        txt_feats = model.encode_text(tokenized)

        # Normalise each embedding
        txt_feats = txt_feats / txt_feats.norm(dim=-1, keepdim=True)

        # Average across prompts
        avg_feat = txt_feats.mean(dim=0)

        # Normalise again (important!)
        avg_feat = avg_feat / avg_feat.norm()

        final_text_features.append(avg_feat.cpu())

# Convert to tensor of shape (num_categories, 512)
final_text_features = torch.stack(final_text_features)
print("Built improved category text embeddings:", final_text_features.shape)


def embed_and_score_clip(img_path):
    img = Image.open(img_path).convert("RGB")
    image_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        # Image embedding
        image_feat = model.encode_image(image_tensor)
        image_feat = image_feat / image_feat.norm(dim=-1, keepdim=True)

        # Cosine similarity to averaged text embeddings
        sims = image_feat @ final_text_features.to(device).T

        # Optional: scale from [-1, 1] → [0, 1]
        sims = (sims + 1) / 2  

    return (
        image_feat.cpu().numpy()[0],   # 512D image embedding
        sims.cpu().numpy()[0]          # independent environmental cue scores
    )


# ------------------------------
# 4. Placeholder stats
# ------------------------------
def empty_attention_stats():
    return {"mean": None, "std": None, "entropy": None, "max_y": None, "max_x": None}

# ------------------------------
# 5. Embed all images
# ------------------------------
for rec in tqdm(point_records[:5000], desc="Embedding points", unit="point"):

    rec["embedding"] = []
    rec["category_scores"] = []
    rec["attn_stats"] = []
    rec["heatmap_paths"] = []

    for img_path in rec["image_files"]:

        # Use ORIGINAL images
        img_path = img_path.replace("airbnb-manchester/", "embeddings/").replace("../", "../../../")

        try:
            embedding, scores = embed_and_score_clip(img_path)

            rec["embedding"].append(embedding)
            rec["category_scores"].append(scores)
            rec["attn_stats"].append(empty_attention_stats())
            rec["heatmap_paths"].append(None)

        except Exception as e:
            tqdm.write(f"⚠️ Error: {e}")

output_file = (data_dir + "sample_points_cache/points_data_cache_with_CLIP_embeddings_and_scores_v3.pkl")

with open(output_file, "wb") as f:
    pickle.dump(point_records, f)

print(f"\n💾 Saved embeddings + category scores for {len(point_records)} points.")

Built improved category text embeddings: torch.Size([11, 512])


Embedding points: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [13:04<00:00,  6.38point/s]



💾 Saved embeddings + category scores for 18897 points.


In [13]:
len(point_records)

18897